# Topic 4: Poisson Process Simulation
## 1: Homogenous Poisson Process

In [7]:
import numpy as np
from scipy.stats import expon

(1) Implement the algorithm from the previous section. Return N(T) as integer, and S1,...SN(T) as an numpy array.

In [8]:
# implement poisson process algorithm
def poisson_process(lmbda, T):
    N = 0
    S = []
    t = 0
    dt = expon.rvs(scale=1 / lmbda)
    while (t + dt < T):
        t += dt
        N += 1
        S += [t]
        dt = expon.rvs(scale=1 / lmbda)
    return N, np.array(S)

(2) Write a function that takes as inputs: T, S1, ...SN(T) and t"< T and return the value of the poisson process at time t. Raise an exception if the user requests t > T. 

In [9]:
# implement function to evaluate poisson process
# at t given T and  arrival times
def evaluate_process(T, S, t):
    if t <= T:
        return np.sum(S <= t)
    else:
        raise Exception('t > T')

(3) Generate 10000 simulations with λ=1.5 and T=5.Calculate the means and covariance the covariance matrix of N(1) - N(0), N(2) - N(1), N(3) - N(2), N(4) - N(3), N(5) - N(4) etc

In [10]:
# generate 10000 simulations, calculate means
# and covariances of N(1) - N(0), N(2) - N(1), etc
lmbda = 1.5
T = 5
nsims = 10000
t_vals = [0,1,2,3,4,5]
Nt = np.zeros((nsims, len(t_vals)))
for i in range(0, nsims):
    N, S = poisson_process(lmbda, T)
    for j, t in enumerate(t_vals):
        Nt[i,j] = evaluate_process(T, S, t)
Nt_diff = Nt[:,1:] - Nt[:,:-1]
print('Poisson process means: ', np.mean(Nt_diff, axis=0))
print('Poisson process covariance: ', np.cov(Nt_diff, rowvar=False))

Poisson process means:  [1.5022 1.499  1.5046 1.5139 1.4912]
Poisson process covariance:  [[ 1.52314747  0.03060526 -0.00831095  0.01322074 -0.0045811 ]
 [ 0.03060526  1.50114911 -0.01159656 -0.0170378  -0.0170105 ]
 [-0.00831095 -0.01159656  1.51953079 -0.01591553 -0.02306183]
 [ 0.01322074 -0.0170378  -0.01591553  1.50635743  0.00517284]
 [-0.0045811  -0.0170105  -0.02306183  0.00517284  1.46986955]]


Implement the algorithm from the previous section so that it returns N(T) and numpy array S given lambda and T

Copy your evaluation function from the previous section over to this notebook

Generate 10000 simulations. For k = 6, 7, 8, 9; plot the empirical pmf of N(t) | N(T) = k for t = 1, 4. Draw a line graph of the theoretical pmf of the corresponding binomial distribution.

In [22]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.stats import poisson, uniform, binom

In [19]:
# generate poisson process
def poisson_process(lmbda, T):
    N = poisson.rvs(lmbda*T)
    S = uniform.rvs(size=N) * T
    return N, np.sort(S)

In [20]:
def evaluate_process(T, S, t):
    if t <= T:
        return np.sum(S <= t)
    else:
        raise Exception('t > T')

In [23]:
# generate 10000 runs of poisson process
lmbda = 1.5
T = 5
nsims = 10000
t_vals = [1, 4]
Nt = np.zeros((nsims, len(t_vals)))
Ns = np.zeros(nsims)
for i in range(0, nsims):
    N, S = poisson_process(lmbda, T)
    Ns[i] = N
    for j, t in enumerate(t_vals):
        Nt[i,j] = evaluate_process(T, S, t)

In [ ]:
#plot conditional distributions from simulations
k_vals = [6,7,8,9]
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(10,10))
for i, k in enumerate(k_vals):
    for j, t in enumerate(t_vals):
        loc_k = (Ns==k)
        Nt_temp = Nt[loc_k,j]
        vals, count = np.unique(Nt_temp, return_counts=True)
        ax[i,j].bar(vals, count/len(Nt_temp))
        x_vals = np.arange(0, k+1)
        theo_vals = binom.pmf(x_vals,  n=k, p = t/T)
        ax[i,j].plot(x_vals, theo_vals)

## 2: Nonhomogeneous Poisson Process

## Thining Method

Implement the thinning Method for a non-homogeneous poisson process, given a function λ(t) with maximum value λmax.

Generate 10,000 paths from a nonhomogeneous poisson process with intensity lambda(t)=−(t−1)^2+2 for 0≤t≤2. Use scipy.optimize.minimize_scalar to find the maximum of this function.

Over the grid t = [.01, .02, ..., 1.0], plot mean number of events that have occured by that time over the set of paths. Compare to to the theoretical integral of λ(t), −t^3/3+t^2+t

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize_scalar
from scipy.stats import poisson, uniform

def poisson_process(lmbda, T):
    N = poisson.rvs(lmbda*T)
    S = uniform.rvs(size=N) * T
    return np.sort(S)

def thinning(lmbda, lmbda_max, T):
    ts = poisson_process(lmbda_max, T)
    S = []
    for t in ts:
        p = lmbda(t) / lmbda_max
        if uniform.rvs() < p:
            S.append(t)
    return np.array(S)

In [3]:
def lmbda(t):
    return -(t-1)**2 + 2

def neg_lmbda(t):
    return -1*lmbda(t)

T = 2

sol = minimize_scalar(neg_lmbda, bounds=(0,T))
lmbda_max = lmbda(sol.x)

n = 10_000

S=[]

for i in range(0, n):
    S.append(thinning(lmbda, lmbda_max, T))

In [4]:
S

[array([0.34300914, 0.36716225, 1.14814991, 1.3385806 , 1.93529639]),
 array([0.79863259, 1.21850425, 1.69920219, 1.75110461]),
 array([0.75680836, 1.38800564]),
 array([1.6006706]),
 array([1.75645632]),
 array([0.31461582, 0.31940497, 0.65277411, 0.82294544, 1.00356685,
        1.57749228, 1.69658684, 1.78284019, 1.98486087]),
 array([1.49943368, 1.60608452]),
 array([0.02335317, 0.44289855, 1.02027236]),
 array([0.49406328, 1.27432213, 1.94210109]),
 array([0.01094654, 0.11997576, 1.28774016, 1.58582714]),
 array([0.06603142, 0.32148491, 0.87262407, 1.11740393, 1.46827181,
        1.62160196]),
 array([0.11210703, 0.5242396 , 1.01347665, 1.27506796, 1.66863128]),
 array([0.01859042, 0.46028622, 0.58985504, 1.52904438]),
 array([0.6793865 , 0.86129855, 1.03140746, 1.40258717, 1.81262275]),
 array([0.32441902, 0.4442172 , 0.86664877, 1.22965292, 1.47607843,
        1.77375081]),
 array([0.17139396, 0.2204528 , 1.28957744, 1.35987834]),
 array([0.79072193, 0.84229665, 1.23161898, 1.392

In [ ]:
ts = np.linspace(0.0, 1, 101)

means = np.zeros(101)

for j, t in enumerate(ts):
    for i in range(0, n):
        means[j] += np.sum(S[i] <= t)

means = means / n

y = -(ts **3)/3 + (ts**2) + ts

plt.plot(ts, means)
plt.plot(ts, y)

## Order Statistic Method

Implement the order statistic method, following the below scaffold for the intensity function:
$\lambda(t)=42t*exp(-t^2)$ for $0<=t<=6$

In [25]:
import numpy as np
from scipy.stats import poisson, uniform

#Compute the expected number of arrivals between 0 and T
#Your function should return a float

def Ex_arrivals(T):
    return 21 * (1- np.exp(-T**2))

#simulate the actual number of arrivals between 0 and T
#hint: you can need to call Ex_arrivals(T) function you 
#constructed in the last step
def N_arrivals(T):
    L = Ex_arrivals(T)
    return poisson.rvs(L)

N_arrivals(T = 6)

#Calculate the inverse cdf for the unordered arrival times

def inv_cdf(u, T):
    term1 = 1 - np.exp(-T**2)
    return np.sqrt(- np.log(1 - u * term1))

# Combine all the previous functions to simulate the process up to time T

np.random.seed(1996)
def Simulate(T):
    N = N_arrivals(T)
    X = np.zeros(N)
    for i in range(N):
        u = uniform.rvs()
        X[i] = inv_cdf(u, T)
    return  np.sort(X)

result_simulation = Simulate(T = 6)
result_simulation

array([0.07162409, 0.250669  , 0.26133756, 0.31571496, 0.40889667,
       0.50773498, 0.50932239, 0.53008839, 0.60658651, 0.62853664,
       0.66058952, 0.76430313, 0.88371247, 0.97831058, 1.06344273,
       1.10980383, 1.18246613, 1.3000786 , 1.30693752, 1.53319418,
       1.81310792, 1.85130138, 1.96527166])

## Inversion Method

Implement the inversion method given generic mean function Λ(t).

Generate some paths using Λ(t)=1−exp(−t^2) from 0≤t≤10.

Generate 10,000 paths from a nonhomogeneous poisson process with mean function Λ(t)=1−exp(−t^2) from 0≤t≤10.

Over the grid t = [.01, .02, ..., 1.0], plot mean number of events that have occured by that time over the set of paths. Compare to the theoretical mean function.

In [27]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import poisson, uniform
from scipy.optimize import root_scalar

def poisson_process(lmbda, T):
    N = poisson.rvs(lmbda*T)
    S = uniform.rvs(size=N) * T
    return np.sort(S)

def Lmbda(t):
    return 1 + np.exp(-t**2)

## implement the inverse method here

def inv_method(Lmbda, T):
    LmbdaT = Lmbda(T)
    s = poisson_process(1, LmbdaT)
    ts = np.zeros(len(s))
    for i, t in enumerate(s):
        L = lambda x: Lmbda(x) - t
        ts[i] = root_scalar(L, bracket=[0,T], method='bisect').root
    return ts

n = 10_000
T = 10

S = []
for i in range(n):
    S.append(inv_method(Lmbda, T))

ts = np.linspace(0.0, 1, 101)

means = np.zeros(101)

for j, t in enumerate(ts):
    for i in range(0, n):
        means[j] += np.sum(S[i] <= t)

means = means / n

y = Lmbda(ts)

plt.plot(ts, means)
plt.plot(ts, y)

ValueError: f(a) and f(b) must have different signs